# Goodfire

Goodfire is an AI inference platform to run certain Llama models with SAE feature steering. See the [Goodfire docs](https://docs.goodfire.ai/) for more information.

In [1]:
import getpass
import os

if "GOODFIRE_API_KEY" not in os.environ:
    os.environ["GOODFIRE_API_KEY"] = getpass.getpass("Enter your Goodfire API key: ")

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [2]:
from langchain_community.chat_models import ChatGoodfire
import goodfire

MODEL_NAME = "meta-llama/Llama-3.3-70B-Instruct"

goodfire_client = goodfire.Client(api_key=os.environ["GOODFIRE_API_KEY"])

base_variant = goodfire.Variant(MODEL_NAME)

enthusiasm_feature = goodfire_client.features.lookup([55543], base_variant)[55543]

enthusiasm_variant = goodfire.Variant(MODEL_NAME)
enthusiasm_variant.set(enthusiasm_feature, 0.3)

llm = ChatGoodfire(
    model=enthusiasm_variant,
    temperature=0.6,
    seed=42,
    # other params...
)

ValueError: model must be a Goodfire variant, got <class 'str'>

## Invocation

In [14]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='J\'ADORE LA PROGRAMMATION! \n\n(or in a more casual tone: J\'ADORE LE CODAGE!)\n\nNote: "J\'adore" is a stronger way to say "I love" in French, it\'s more like "I\'m crazy about" or "I\'m absolutely passionate about". If you want to use a more literal translation, you can say: "J\'aime la programmation" which means "I like programming".', additional_kwargs={}, response_metadata={}, id='run-d91dd50b-1d6a-4c04-a78c-b1b922c1fc92-0')

Note: The variant can be overridden after instantiation by providing a new variant to the `model` parameter.

In [17]:
ai_msg = llm.invoke(messages, model=base_variant)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={}, id='run-b646d8ed-74c3-40a2-8530-7f094060bf23-0')

In [15]:
print(ai_msg.content)

J'ADORE LA PROGRAMMATION! 

(or in a more casual tone: J'ADORE LE CODAGE!)

Note: "J'adore" is a stronger way to say "I love" in French, it's more like "I'm crazy about" or "I'm absolutely passionate about". If you want to use a more literal translation, you can say: "J'aime la programmation" which means "I like programming".


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={}, response_metadata={}, id='run-f77167ac-e9a8-4fc0-9e43-5a4800290324-0')

## API reference

For detailed documentation of all ChatGoodfire features and configurations head to the API reference: https://python.langchain.com/api_reference/goodfire/chat_models/langchain_goodfire.chat_models.ChatGoodfire.html
